<a href="https://colab.research.google.com/github/luigiselmi/dl_tensorflow/blob/main/residual_connections.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Residual connections
Deep neural networks are affected by the *vanishing gradients* problem for which the parameters become increasingly smaller. One solution to avoid such problem is to use a shortcut, that is to add the input to the output of a block of one or more layers.

In [1]:
from tensorflow import keras
from tensorflow.keras import layers

In [5]:
inputs = keras.Input(shape=(32, 32, 3))
x = layers.Conv2D(filters=32, kernel_size=3, activation="relu")(inputs)
residual = x
residual.shape

TensorShape([None, 30, 30, 32])

In [6]:
x = layers.Conv2D(64, 3, activation="relu", padding="same")(x)
x.shape

TensorShape([None, 30, 30, 64])

In [7]:
residual = layers.Conv2D(64, 1)(residual)
residual.shape

TensorShape([None, 30, 30, 64])

In [8]:
x = layers.add([x, residual])
x.shape

TensorShape([None, 30, 30, 64])